In [7]:
from bokeh.io import show, output_file
from bokeh.plotting import figure, output_file, show
import numpy as np

In [8]:
from bokeh.models import ColumnDataSource, Plot,DatetimeTickFormatter,FixedTicker

In [9]:
Datapath_f1 = 'C:\\Users\\z0043H7B\\Tien\\Data\\dubuque\\wacker\\exploreSPM\\'

In [10]:
import sys
sys.path.insert(0, Datapath_f1)

In [11]:
import Tools10
import numpy as np
import time
import os
import pandas as pd
%load_ext autoreload
%autoreload 10

In [12]:
# test use a time series figure
Datapath = '/Users/z0043H7B/Tien/Data/dubuque/Dubuque_080819/Wacker/'

In [13]:
import os
os.chdir(Datapath)

In [14]:
newdf=pd.read_csv('Wacker_15minDetAct.csv',encoding='utf-8',parse_dates=True,index_col=0)

In [15]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14208 entries, 2018-10-17 10:45:00 to 2019-03-14 10:30:00
Data columns (total 16 columns):
det33          14208 non-null int64
det35          14208 non-null int64
det37          14208 non-null int64
det39          14208 non-null int64
det41          14208 non-null int64
det43          14208 non-null int64
det45          14208 non-null int64
det47          14208 non-null int64
totalct        14208 non-null int64
month          14208 non-null int64
day            14208 non-null int64
hour           14208 non-null int64
minute         14208 non-null int64
day-of-week    14208 non-null int64
year           14208 non-null int64
min_bin        14208 non-null int64
dtypes: int64(16)
memory usage: 1.8 MB


In [16]:
# dump out one time sereis:
data1 = newdf.totalct.values

In [98]:
# preplot the time series
x = np.linspace(0,len(data1),len(data1))

In [23]:
output_file('testplt_tcnt.html')
# setup size 
p = figure(title='total count',plot_width=800, plot_height=400)

In [25]:
p.circle(x,data1, color='blue', size=7)

GlyphRenderer(id='1040', ...)

In [26]:
show(p)

In [107]:
from scipy.fftpack import fft, ifft
ret = fft(data1)

In [108]:
ret[0:10]

array([ 1.48053200e+06   +0.j        ,  2.16566597e+04+6867.53465515j,
       -6.97942427e+03-2413.64348393j,  8.81387737e+03+6561.90262585j,
       -9.66163871e+03-6101.68835998j,  1.00694880e+03+2685.25423693j,
       -4.00724998e+02+3777.21151486j,  5.13299171e+03-2351.56735972j,
       -1.62072097e+03+2626.32292463j,  1.55104751e+03+1643.95534041j])

In [109]:
Y=ret[range(int(len(data1)/2))]

In [110]:
# construct frequency
Fs = 1/(15*60)  # sample rate
Ts = 15*60

In [111]:
freq = np.arange(len(data1))*Fs/len(data1)
freq = freq[range(int(len(data1)/2))]

In [115]:
from bokeh.models import HoverTool
output_file('testplt_fftfqcAmp.html')
# setup size 
p = figure(title='fft frequencies',plot_width=800, plot_height=400, x_axis_label='Freq(Hz)',y_axis_label='Amplitude')
p.line(freq, abs(Y) , line_color='red', line_width=2)
p.line(freq, newY , line_color='green', line_width=2)

#p.toolbar.active_inspect = [hover_tool, crosshair_tool]
x=freq
y=abs(Y)
p.add_tools(HoverTool(
    tooltips=[
        ( 'freq',   '$x'            ),
        ( 'Amp',  '$y' ), # use @{ } for field names with spaces
    ],

    formatters={
        'freq'      : 'printf', # use 'datetime' formatter for 'date' field
        'Amp' : 'printf',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },
))

show(p)

In [113]:
#smooth out some noises on the frequencies
newY = [x if x > 1.5e4 else 0 for x in abs(Y)]
# mask out the high frequencies
# presevere non-zero amplitudes but keep the phase


In [114]:
# reconstruct the signal using ifft
# extract the imaginary part
imag=np.concatenate((np.imag(Y),np.imag(Y[::-1])), axis=0)
real=np.concatenate((newY,newY[::-1]),axis=0)
full = imag*1j+real
recst=ifft(full)

In [95]:
full[0:10]

array([104.20411036+0.j        ,   1.59906122+0.48335689j,
         0.        -0.16987919j,   0.        +0.46184562j,
         0.        -0.42945442j,   0.        +0.18899593j,
         0.        +0.26585104j,   0.        -0.16551009j,
         0.        +0.18484818j,   0.        +0.11570632j])

In [100]:
# try plotting inversed fft signal
output_file('testplt_ifft.html')
# setup size 
p = figure(title='ifft signal',plot_width=800, plot_height=400, x_axis_label='time(15mins)',y_axis_label='car counts')
p.circle(x,data1, color='blue', size=7)
p.circle(x,abs(recst), color='green', size=7)

#p.toolbar.active_inspect = [hover_tool, crosshair_tool]
x=x
y=abs(full)
p.add_tools(HoverTool(
    tooltips=[
        ( 'time',   '$x'            ),
        ( 'car counts',  '$y' ), # use @{ } for field names with spaces
    ],

    formatters={
        'time'      : 'printf', # use 'datetime' formatter for 'date' field
        'car counts' : 'printf',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },
))

show(p)

In [87]:
len(k)

14208